In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
os.getenv('PGDB_USER')

'dnb'

In [3]:
%run -i '../pyscript/db/main.py'

schema is dnb port is 5432 database is dnb


In [4]:
from sqlalchemy import create_engine, text

In [5]:
sql = 'select * from information_schema.tables where table_schema = \'daily_stats\';'

df_tables = db.query_df(sql)

In [6]:
sql = 'select * from information_schema.columns where table_schema = \'daily_stats\';'

df_columns = db.query_df(sql)

In [15]:
sql = "select column_name from information_schema.columns where table_name ilike 'dc%' and column_name ilike '%_nom' and table_name ilike '%_nr_%' order by column_name;"
df_kpi = db.query_df(sql)
kpis = df_kpi.values.tolist()   

In [16]:
kpis.sort()
kpis

[['5g_ho_success_rate_dnb_5g_to_dnb_nom'],
 ['5g_ho_success_rate_dnb_5g_to_dnb_nom'],
 ['average_cqi_nom'],
 ['average_cqi_nom'],
 ['avg_pusch_ul_rssi_nom'],
 ['avg_pusch_ul_rssi_nom'],
 ['cell_availability_nom'],
 ['cell_availability_nom'],
 ['dl_16qam_nom'],
 ['dl_16qam_nom'],
 ['dl_16qam_nom'],
 ['dl_16qam_nom'],
 ['dl_256qam_nom'],
 ['dl_256qam_nom'],
 ['dl_256qam_nom'],
 ['dl_256qam_nom'],
 ['dl_64qam_nom'],
 ['dl_64qam_nom'],
 ['dl_64qam_nom'],
 ['dl_64qam_nom'],
 ['dl_cell_throughput_nom'],
 ['dl_cell_throughput_nom'],
 ['dl_data_volume_gb_nom'],
 ['dl_data_volume_gb_nom'],
 ['dl_mac_vol_as_scell_ext_nom'],
 ['dl_mac_vol_as_scell_ext_nom'],
 ['dl_mac_vol_as_scell_nom'],
 ['dl_mac_vol_as_scell_nom'],
 ['dl_mac_vol_to_scell_ext_nom'],
 ['dl_mac_vol_to_scell_ext_nom'],
 ['dl_mac_vol_to_scell_nom'],
 ['dl_mac_vol_to_scell_nom'],
 ['dl_qpsk_nom'],
 ['dl_qpsk_nom'],
 ['dl_qpsk_nom'],
 ['dl_qpsk_nom'],
 ['dl_user_throughput_nom'],
 ['dl_user_throughput_nom'],
 ['dl_user_throughput_nom'

In [18]:
# join all elements in kpis separated by commas
kpis_string = ','.join(['"'+ elem[0]+'"' for elem in kpis])


In [19]:
kpis_string

'"5g_ho_success_rate_dnb_5g_to_dnb_nom","5g_ho_success_rate_dnb_5g_to_dnb_nom","average_cqi_nom","average_cqi_nom","avg_pusch_ul_rssi_nom","avg_pusch_ul_rssi_nom","cell_availability_nom","cell_availability_nom","dl_16qam_nom","dl_16qam_nom","dl_16qam_nom","dl_16qam_nom","dl_256qam_nom","dl_256qam_nom","dl_256qam_nom","dl_256qam_nom","dl_64qam_nom","dl_64qam_nom","dl_64qam_nom","dl_64qam_nom","dl_cell_throughput_nom","dl_cell_throughput_nom","dl_data_volume_gb_nom","dl_data_volume_gb_nom","dl_mac_vol_as_scell_ext_nom","dl_mac_vol_as_scell_ext_nom","dl_mac_vol_as_scell_nom","dl_mac_vol_as_scell_nom","dl_mac_vol_to_scell_ext_nom","dl_mac_vol_to_scell_ext_nom","dl_mac_vol_to_scell_nom","dl_mac_vol_to_scell_nom","dl_qpsk_nom","dl_qpsk_nom","dl_qpsk_nom","dl_qpsk_nom","dl_user_throughput_nom","dl_user_throughput_nom","dl_user_throughput_nom","dl_user_throughput_nom","e-rab_block_rate_nom","e-rab_block_rate_nom","endc_ca_configuration_sr_nom","endc_ca_configuration_sr_nom","endc_ca_deconfigur

In [10]:
kpi_dict = {}

for table in df_columns['table_name'].unique():
    kpi_dict[table] = {}
    print()
    print(f'==========={table}==========')
    columns = df_columns[df_columns['table_name'] == table]['column_name'].unique()
    for column in columns:
        if (column.endswith('_nom')):
            nom = column
            kpi = column[:-4].strip()
            # find the corresponding denom
            denom = column[:-4] + '_den'
            # check denom column exists
            if denom in columns:
                # raise Exception(f'No denom column for {kpi}, nom is {nom}')
                # calculate kpi
                kpi_dict[table][kpi] = f'sum("{nom}")|||sum("{denom}")'
                print(kpi_dict[table][kpi], ' as ', '"'+kpi+'"', ',')
            else:
                kpi_dict[table][kpi] = f'sum("{nom}")'
                print(kpi_dict[table][kpi], ' as ', '"'+kpi+'"', ' ,')
        


===========dc_e_nr_events_nrcelldu_flex_day==========
sum("ul_traffic_volume_nom")|||sum("ul_traffic_volume_den")  as  "ul_traffic_volume" ,
sum("dl_qpsk_nom")  as  "dl_qpsk"  ,
sum("dl_16qam_nom")  as  "dl_16qam"  ,
sum("dl_64qam_nom")  as  "dl_64qam"  ,
sum("dl_256qam_nom")  as  "dl_256qam"  ,
sum("ul_qpsk_nom")  as  "ul_qpsk"  ,
sum("ul_16qam_nom")  as  "ul_16qam"  ,
sum("ul_64qam_nom")  as  "ul_64qam"  ,
sum("dl_user_throughput_nom")|||sum("dl_user_throughput_den")  as  "dl_user_throughput" ,
sum("ul_user_throughput_nom")|||sum("ul_user_throughput_den")  as  "ul_user_throughput" ,

===========dc_e_nr_nrcelldu_day==========
sum("dl_user_throughput_nom")|||sum("dl_user_throughput_den")  as  "dl_user_throughput" ,
sum("ul_user_throughput_nom")|||sum("ul_user_throughput_den")  as  "ul_user_throughput" ,
sum("dl_cell_throughput_nom")|||sum("dl_cell_throughput_den")  as  "dl_cell_throughput" ,
sum("ul_cell_throughput_nom")|||sum("ul_cell_throughput_den")  as  "ul_cell_throughput" ,
sum(

# create tables

In [8]:
dpm_path = '/home/hawkuser/data2/var/www/hawk-express-app/db/df_dpm/dnb_2023-4-26_6-53-52.json'

df_dpm = pd.read_json(dpm_path)
df_dpm.head()


,dnb_index,nominal_id,_id,Nominal_ID,Site_Name,Latitude,Longitude,site_id,Nominal_Latitude,Nominal_Longitude,Candidate_Latitude,Candidate_Longitude,on_board_date,api_call_date,added,Acceptance_Cluster,Sub_Cluster,CBOClusterName
0,DWPPJ8002,PJYIBC_003,62373ac01b9008e7fa6e8485,PJYIBC_003,DWPPJ8002_MOFPUTRAJAYA,NaN,101.68882,DWPPJ8002,2.92584,101.68841,2.92584,101.68800,2021-12-13 00:00:00,2023-04-26 04:41:37.886391,No,,,
1,DWKUL8001,KULIBC_001,6265fd739ba5745c7db16b2b,KULIBC_001,DWKUL8001_KLSENTRALSTATION,NaN,NaN,DWKUL8001,3.13420,101.68610,3.13420,101.68610,2022-08-16 00:00:00,2023-04-26 04:41:37.886391,No,,,
2,DWKUL8005,KULIBC_002,62373ac11b9008e7fa6e84a5,KULIBC_002,DWKUL8501_TRXEXCHANGE106,NaN,101.71870,DWKUL8501,3.14120,101.71810,3.14180,101.71870,2022-01-13 00:00:00,2023-04-26 04:41:37.886391,No,,,
3,DWKUL8504,KULIBC_005,6265fd739ba5745c7db16b2d,KULIBC_005,DWKUL8504_LOT10SHOPPINGCENTRE,NaN,101.71197,DWKUL8504,3.14662,101.71162,3.14630,101.71197,2022-02-18 00:00:00,2023-04-26 04:41:37.886391,No,,,
4,DBSEP1810,SGR_1006,62373a9c1b9008e7fa6e820b,SGR_1006,DBSEP1810_CBD2PSRNMULTIMEDIA,NaN,NaN,DBSEP1810,2.92583,101.65080,2.92606,101.65117,2021-12-30 00:00:00,2023-04-26 04:41:37.886391,No,None,None,


In [10]:
db.df_to_db(df_dpm, schema='daily_stats', name='df_dpm', if_exists='replace', index=False)

In [10]:
cell_mapping_path = '/home/hawkuser/data2/var/www/hawk-express-app/db/df_dpm/cell_mapping_2023-4-26_14-12-24.json'

df_cell_mapping = pd.read_json(cell_mapping_path)
df_cell_mapping.head()


,Cellname,Region,Cluster_ID,DISTRICT,MCMC_State,geom,SITEID,SystemID,Sitename
0,DAKIN0154_N3_0310,NORTHERN,PKIN_01,KINTA,PERAK,0101000020E6100000E17A14AE4745594033C34659BFB9...,DAKIN0154,N3,None
1,DAKIN0154_N3_0320,NORTHERN,PKIN_01,KINTA,PERAK,0101000020E6100000E17A14AE4745594033C34659BFB9...,DAKIN0154,N3,None
2,DAKIN0154_N3_0330,NORTHERN,PKIN_01,KINTA,PERAK,0101000020E6100000E17A14AE4745594033C34659BFB9...,DAKIN0154,N3,None
3,DAKIN0156_L7_0010,NORTHERN,PKIN_07,KINTA,PERAK,0101000020E6100000CD920035B5465940F33CB83B6B57...,DAKIN0156,L7,DAKIN0156_13JLNMARZUKI
4,DAKIN0156_L7_0020,NORTHERN,PKIN_07,KINTA,PERAK,0101000020E6100000CD920035B5465940F33CB83B6B57...,DAKIN0156,L7,DAKIN0156_13JLNMARZUKI


In [11]:
db.df_to_db(df_cell_mapping, schema='daily_stats', name='cell_mapping', if_exists='replace', index=False)